In [ ]:
# 6.3.2
import numpy as np
import torch
import torch.nn as nn

def custom_batch_norm1d(input_tensor, eps):
    m = input_tensor.mean(dim=0, keepdim=True)
    sd = input_tensor.var(dim=0, keepdim=True,unbiased=False)
    z = (input_tensor - m) / ((sd + eps).sqrt())
    beta = 0
    gamma = 1
    normed_tensor = z * gamma + beta
    return normed_tensor


input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# import numpy as np
# all_correct = True
# for eps_power in range(10):
#     eps = np.power(10., -eps_power)
#     batch_norm.eps = eps
#     batch_norm_out = batch_norm(input_tensor)
#     custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)

#     all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
#     all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape

True
tensor([[ 0., -1.,  0.,  0., -1.],
        [ 0.,  1.,  0.,  0.,  1.]])
tensor([[ 0., -1.,  0.,  0., -1.],
        [ 0.,  1.,  0.,  0.,  1.]])


In [ ]:
# 6.3.3
import torch
import torch.nn as nn

input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    m = input_tensor.mean(dim=0, keepdim=True)
    sd = input_tensor.var(dim=0, keepdim=True,unbiased=False)
    z = (input_tensor - m) / ((sd + eps).sqrt())
    beta = bias
    gamma = weight
    normed_tensor = z * gamma + beta
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm_out = batch_norm(input_tensor)
custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
print(torch.allclose(batch_norm_out, custom_batch_norm_out) \
      and batch_norm_out.shape == custom_batch_norm_out.shape)
print(batch_norm_out)
print(custom_batch_norm_out)

True
tensor([[ 0.4252,  1.9116,  2.6790, -0.1251,  1.7450, -0.4883,  0.2580],
        [ 0.8117,  0.2487, -2.2819, -0.5931, -0.6168,  0.0985,  0.3230],
        [ 0.2299, -1.6845,  0.6501, -1.0799, -1.0488, -0.6731, -0.9794],
        [ 0.4329,  3.6908,  0.5654, -2.1456, -1.1317,  0.6511, -0.1192],
        [ 0.4233,  0.1114,  1.6985, -2.9717, -0.8786, -0.5284, -0.5487]],
       grad_fn=<NativeBatchNormBackward0>)
tensor([[ 0.4252,  1.9116,  2.6790, -0.1251,  1.7450, -0.4883,  0.2580],
        [ 0.8117,  0.2487, -2.2819, -0.5931, -0.6168,  0.0985,  0.3230],
        [ 0.2299, -1.6845,  0.6501, -1.0799, -1.0488, -0.6731, -0.9794],
        [ 0.4329,  3.6908,  0.5654, -2.1456, -1.1317,  0.6511, -0.1192],
        [ 0.4233,  0.1114,  1.6985, -2.9717, -0.8786, -0.5284, -0.5487]])


In [ ]:
# 6.3.4
import torch
import torch.nn as nn


input_size = 3
batch_size = 5
eps = 1e-1


class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        self.running_mean = 0
        self.running_var = 1
        self.is_eval = False

    def __call__(self, input_tensor):
      if not self.is_eval:
        m = input_tensor.mean(dim=0, keepdim=True)
        sd = input_tensor.var(dim=0, keepdim=True,unbiased=False)
        z = (input_tensor - m) / ((sd + self.eps).sqrt())
        self.running_mean = (1 - self.momentum) * m + self.momentum * self.running_mean
        self.running_var = (1 - self.momentum) * input_tensor.var(dim=0, keepdim=True,unbiased=True) + self.momentum * self.running_var
      else:
        z = (input_tensor - self.running_mean) / ((self.running_var + self.eps).sqrt())
      normed_tensor = z * self.weight + self.bias
      return normed_tensor

    def eval(self):
      self.is_eval = True
        


batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

print(all_correct)

True


In [ ]:
# 6.3.5
import torch
import torch.nn as nn

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
    normed_tensor = torch.zeros(input_tensor.shape)
    for j in range(input_tensor.shape[1]):
      m = torch.mean(input_tensor[:,j,:,:])
      sd = torch.var(input_tensor[:,j,:,:],unbiased=False)
      z = (input_tensor[:,j,:,:] - m) / ((sd + eps).sqrt())
      beta = 0
      gamma = 1
      normed_tensor[:,j,:,:] = z * gamma + beta
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)



# print(input_tensor)
# print(norm_output)
# print(input_tensor.shape)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


In [ ]:
# 6.3.6
import torch
import torch.nn as nn


eps = 1e-10


def custom_layer_norm(input_tensor, eps):
    normed_tensor = torch.zeros(input_tensor.shape)
    for j in range(input_tensor.shape[0]):
      m = torch.mean(input_tensor[j])
      sd = torch.var(input_tensor[j],unbiased=False)
      z = (input_tensor[j] - m) / ((sd + eps).sqrt())
      beta = 0
      gamma = 1
      normed_tensor[j] = z * gamma + beta
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):

all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    print(input_tensor.shape)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

torch.Size([3, 4])
torch.Size([3, 4, 5])
torch.Size([3, 4, 5, 6])
torch.Size([3, 4, 5, 6, 7])
torch.Size([3, 4, 5, 6, 7, 8])
torch.Size([3, 4, 5, 6, 7, 8, 9])
True


In [ ]:
# 6.3.7
import torch
import torch.nn as nn

eps = 1e-3

batch_size = 5
input_channels = 2
input_length = 30

instance_norm = nn.InstanceNorm1d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, input_length, dtype=torch.float)


def custom_instance_norm1d(input_tensor, eps):
    normed_tensor = torch.zeros(input_tensor.shape)
    for i in range(input_tensor.shape[0]):
      for j in range(input_tensor.shape[1]):
        m = torch.mean(input_tensor[i,j, :])
        sd = torch.var(input_tensor[i,j, :],unbiased=False)
        z = (input_tensor[i,j, :] - m) / ((sd + eps).sqrt())
        beta = 0
        gamma = 1
        normed_tensor[i,j, :] = z * gamma + beta
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = instance_norm(input_tensor)
custom_output = custom_instance_norm1d(input_tensor, eps)
print(norm_output.shape)
print(custom_output.shape)
print(torch.allclose(norm_output, custom_output, atol=1e-06) and norm_output.shape == custom_output.shape)

torch.Size([5, 2, 30])
torch.Size([5, 2, 30])
True


In [ ]:
# 6.3.8
import torch
import torch.nn as nn

channel_count = 6
eps = 1e-3
batch_size = 20
input_size = 2

input_tensor = torch.randn(batch_size, channel_count, input_size)


def custom_group_norm(input_tensor, groups, eps):
    normed_tensor = torch.zeros(input_tensor.shape)
    step = int(input_tensor.shape[1]/groups)
    print(step)
    for i in range(input_tensor.shape[0]):
      for j in range(0,input_tensor.shape[1], step):
        m = torch.mean(input_tensor[i,j:j+step])
        sd = torch.var(input_tensor[i,j:j+step], unbiased=False)
        z = (input_tensor[i,j:j+step] - m) / ((sd + eps).sqrt())
        beta = 0
        gamma = 1
        normed_tensor[i,j:j+step] = z * gamma + beta
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for groups in [1, 2, 3, 6]:
    group_norm = nn.GroupNorm(groups, channel_count, eps=eps, affine=False)
    norm_output = group_norm(input_tensor)
    custom_output = custom_group_norm(input_tensor, groups, eps)
    all_correct &= torch.allclose(norm_output, custom_output, 1e-3)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

6
3
2
1
True
